<a href="https://colab.research.google.com/github/rrfsantos/Large-Language-Model-Agents-MOOC-Fall-2024/blob/main/LAB01/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -r /content/drive/MyDrive/LLM-Lab1/requirements.txt

In [ ]:
# -*- coding: utf-8 -*-
from autogen import ConversableAgent, UserProxyAgent, GroupChat, GroupChatManager, register_function
from numpy import sqrt
from typing import Dict, List
import sys
import os
from dotenv import load_dotenv
import re

# Load environment variables from .env file
load_dotenv()


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
#--------------------------------------------------------- AGENT FUNCTIONS ------------------------------------------------------------#

In [5]:
def normalize_name(name: str) -> str:
    # Remove special characters and convert to lowercase
    return re.sub(r'[^a-zA-Z0-9]', '', name).lower()

def fetch_restaurant_data(restaurant_name: str) -> Dict[str, List[str]]:
    normalized_input_name = normalize_name(restaurant_name)
    restaurant_reviews = {}
    original_names = {}

    with open('/content/drive/MyDrive/LLM-Lab1/restaurant-data.txt', 'r') as file:
        for line in file:
            name, review = line.split('. ', 1)
            name = name.strip()
            review = review.strip()
            normalized_name = normalize_name(name)
            if normalized_name not in restaurant_reviews:
                restaurant_reviews[normalized_name] = []
                original_names[normalized_name] = name
            restaurant_reviews[normalized_name].append(review)

    true_name = original_names.get(normalized_input_name, restaurant_name)
    return {true_name: restaurant_reviews.get(normalized_input_name, [])}

fetch_restaurant_data("Taco bell")

{'Taco Bell': ['The food at Taco Bell was bad, with flavors that seemed artificial. The customer service was average, neither particularly helpful nor rude.',
  'The food was bad, with flavors that were off and textures that were strange. The service was average, neither particularly good nor terrible.',
  'The food was bad and left me feeling queasy. The customer service was forgettable, neither good nor terrible.',
  'The food at Taco Bell was bad, with flavors that seemed artificial and unappealing. The service was average, neither particularly good nor terrible.',
  'The food was bad, but the service was average. The tacos were unpleasant and lacked flavor, though the staff was polite enough.',
  'The food at Taco Bell was bad, with flavors that seemed artificial. The service was average, neither impressive nor terrible.',
  'The food at Taco Bell was bad, with flavors that felt artificial and unappetizing. The customer service was average, neither adding to nor detracting from the

In [6]:
def calculate_overall_score(restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]) -> Dict[str, float]:
    if len(food_scores) != len(customer_service_scores):
        raise ValueError("The length of food_scores and customer_service_scores must be the same.")

    N = len(food_scores)
    total_score = 0.0
    geometric_mean = 0.0

    for i in range(N):
        geometric_mean = sqrt(food_scores[i]**2 * customer_service_scores[i])
        total_score += geometric_mean

    overall_score = (total_score / (N * sqrt(125))) * 10

    return {restaurant_name: "{:.3f}".format(overall_score)}

In [7]:
#------------------------------------------------------------ MAIN FUNCTION --------------------------------------------------------#

In [8]:
# Defina a chave da API como variável de ambiente
os.environ["OPENAI_API_KEY"] =

In [9]:
def main(user_query: str):

    # Load environment variables from .env file
    llm_config = {"config_list": [{"model": "gpt-4o", "api_key" : os.getenv("OPENAI_API_KEY")}]}

# ------------------------------------------------------- AGENT'S SYSTEM MESSAGE ----------------------------------------------------#

    entrypoint_agent_system_message = """Analyze the user query {user_query} and use your knowlegde to promptly return the restaurant name in lowercase.
    Send the restaurant name to Data Fetcher"""

    data_fetch_agent_system_message = """ The Data Fetcher.
        Solve the task using the python function fetch_restaurant_data. You use the fetch_restaurant_data function exclusively.
        1. Use the correct restaurant name as input for the fetch_restaurant_data function.
        2. Stop the search as soon as you find the correct match name in the file object.
        3. Your final message must be the output containing correct restaurant's reviews generated by fetch_restaurant_data
        function in the format -> Dict[str, List[str]].
        4. Send the output to Review retriever.
        """

    review_agent_system_massage = """ The Review Retriever.
        Analyze the reviews for the restaurant. You must find two scores: `food_scores` rates the food quality from 1 to 5, and `customer_service_scores` rates the customer service quality from 1 to 5.
        You should find these scores by looking for specific words in the review. Each review has words that match the scores for `food_score` and `customer_service_scores`.
        These are the words to look for:
          - Score 1/5: awful, horrible, or disgusting
          - Score 2/5: bad, unpleasant, or offensive
          - Score 3/5: average, uninspiring, or forgettable
          - Score 4/5: good, enjoyable, or satisfying
          - Score 5/5: awesome, incredible, or amazing

          Each review will have exactly two of these words, one for the food and one for the customer service.
          Important:The score (N/5) is only based on these words. No other factors should determine the score.
          Generate two list food_scores, customer_service_scores.
          Ensure the following:
          - The lists must have the same length.
          - Each review has exactly two scores: one food score and one customer service score.
          - The number of food and customer service scores must exactly match the number of reviews.
          - You must score all the reviews.

        **** Is mandatory to send the information to the Scoring Calculator with the format below:
            Arguments: {restaurant_name: str, food_scores: List[int], customer_service_scores: List[int]} ****
        """

    score_agent_system_message = """ The Score Calculator.
        Solve the task using the python function calculate_overall_score.
        You use the calculate_overall_score function exclusively.
        Your last message MUST have the format: {restaurant_name: round(overall_score, 3)} --->>>>
        """

    #-------------------------------------------------- AGENT'S CONFIGURATION ------------------------------------------------------#

    initializer = ConversableAgent(name="Init",
                                   human_input_mode="NEVER",
                                   code_execution_config=False)

    entrypoint_agent = ConversableAgent(name="Supervisor",
                                        human_input_mode="NEVER",
                                        system_message=entrypoint_agent_system_message,
                                        llm_config=llm_config,
                                        code_execution_config=False)

    data_fetch_agent = UserProxyAgent(name="Data Fetcher",
                                        system_message=data_fetch_agent_system_message,
                                        llm_config=False,
                                        human_input_mode="NEVER",
                                        code_execution_config = {"use_docker": False})

    review_agent = ConversableAgent(name="Review Retriever",
                                    system_message=review_agent_system_massage,
                                    llm_config=llm_config,
                                    human_input_mode="NEVER",
                                    code_execution_config=False)

    score_agent = ConversableAgent(name = "Score Calculator",
                                   system_message=score_agent_system_message,
                                   llm_config=False,
                                   human_input_mode="NEVER",
                                   code_execution_config = {"use_docker": False})

    # ------------------------------------------------ Register the function to the two agents ----------------------------------------------#


    register_function(fetch_restaurant_data,
        caller=entrypoint_agent,  # The assistant agent can suggest calls to the calculator.
        executor=data_fetch_agent,  # The user proxy agent can execute the calculator calls.
        name="fetch_restaurant_data",  # By default, the function name is used as the tool name.
        description="Fetches the reviews for a specific restaurant.")  # A description of the tool.)

    register_function(calculate_overall_score,
        caller=review_agent,  # The assistant agent can suggest calls to the calculator.
        executor=score_agent,  # The user proxy agent can execute the calculator calls.
        name="calculate_overall_score",  # By default, the function name is used as the tool name.
        description="Calculate restaurants's overall scores based on food scores and customer service scores")  # A description of the tool.)"""


    # -------------------------------------------------------- STATEFLOW ---------------------------------------------------------------#

    def state_transition(last_speaker, groupchat):


      if last_speaker is initializer:
          # init -> retrieve
          return entrypoint_agent
      elif last_speaker is entrypoint_agent:
          # retrieve: action 1 -> action 2
          return data_fetch_agent
      elif last_speaker is data_fetch_agent:
          return review_agent
      elif last_speaker is review_agent:
          return score_agent
              # retrieve --(execution failed)--> retrieve


    #--------------------------------------------------------------- GROUPCHAT -----------------------------------------------------#

    groupchat = GroupChat(
        agents=[initializer, entrypoint_agent, data_fetch_agent, review_agent, score_agent],
        messages=[],
        max_round=20,
        speaker_selection_method = state_transition,
        send_introductions=True,
        allow_repeat_speaker=True)

    manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)
    result = initializer.initiate_chat(manager, message=user_query)

    return result

In [10]:
# DO NOT modify this code below.
if __name__ == "__main__":
    assert len(sys.argv) > 1, "Please ensure you include a query for some restaurant when executing main."
    #main(sys.argv[1])
    main("What is the overall score for Taco Bell?")

[autogen.oai.client: 10-23 23:21:51] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-23 23:21:52] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-23 23:21:52] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-23 23:21:52] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 10-23 23:21:52] {184} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


Init (to chat_manager):

What is the overall score for Taco Bell?

--------------------------------------------------------------------------------

Next speaker: Supervisor

Supervisor (to chat_manager):

***** Suggested tool call (call_iVzEZDygQ8jWZPRuE8GIDynM): fetch_restaurant_data *****
Arguments: 
{"restaurant_name":"taco bell"}
**************************************************************************************

--------------------------------------------------------------------------------

Next speaker: Data Fetcher


>>>>>>>> EXECUTING FUNCTION fetch_restaurant_data...
Data Fetcher (to chat_manager):

Data Fetcher (to chat_manager):

***** Response from calling tool (call_iVzEZDygQ8jWZPRuE8GIDynM) *****
{"Taco Bell": ["The food at Taco Bell was bad, with flavors that seemed artificial. The customer service was average, neither particularly helpful nor rude.", "The food was bad, with flavors that were off and textures that were strange. The service was average, neither part